In [1]:
!nvidia-smi

Tue Oct 15 03:44:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[setencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 1.0.1
Uninstalling accelerate-1.0.1:
  Successfully uninstalled accelerate-1.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 32.7 MB/s eta 0:00:00
Using cached accelerate-1.0.1-py3-none-any.whl (330 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [4]:
# Install necessary packages for Google Colab
!pip install transformers datasets nltk tqdm torch evaluate

# Importing necessary libraries
from transformers import pipelines, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
from evaluate import load as load_metric  # Changed load_metric to use evaluate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Importing NLTK for tokenization
import nltk
from nltk.tokenize import sent_tokenize

# Progress bar utility
from tqdm import tqdm

# PyTorch library for using models with transformers
import torch

# Download NLTK punkt tokenizer
nltk.download("punkt")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device="cuda"if torch.cuda.is_available() else "cpu"

model_ckpt="google/pegasus-cnn_dailymail"

tokenizer= AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [6]:
# Download the file
!wget -O Dataset_text_summarization.zip https://github.com/Sujal786Ninawe/Text_Summarization/blob/444be31aec504ba644485b8db7d2347636e48e0f/Dataset_text_summarization.zip?raw=true


--2024-10-15 03:46:24--  https://github.com/Sujal786Ninawe/Text_Summarization/blob/444be31aec504ba644485b8db7d2347636e48e0f/Dataset_text_summarization.zip?raw=true
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Sujal786Ninawe/Text_Summarization/raw/444be31aec504ba644485b8db7d2347636e48e0f/Dataset_text_summarization.zip [following]
--2024-10-15 03:46:24--  https://github.com/Sujal786Ninawe/Text_Summarization/raw/444be31aec504ba644485b8db7d2347636e48e0f/Dataset_text_summarization.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Sujal786Ninawe/Text_Summarization/444be31aec504ba644485b8db7d2347636e48e0f/Dataset_text_summarization.zip [following]
--2024-10-15 03:46:24--  https://raw.githubusercontent.com/Sujal786Ninawe/Text_Summarization/444be31ae

In [7]:
!unzip /content/Dataset_text_summarization.zip

Archive:  /content/Dataset_text_summarization.zip
  inflating: CSV/hiddentest_dialogue.csv  
  inflating: CSV/hiddentest_topic.csv  
  inflating: CSV/test.csv            
  inflating: CSV/train.csv           
  inflating: CSV/validation.csv      
  inflating: JSONL/hiddentest_dialogue.jsonl  
  inflating: JSONL/hiddentest_topic.jsonl  
  inflating: JSONL/test.jsonl        
  inflating: JSONL/train.jsonl       
  inflating: JSONL/validation.jsonl  


In [8]:
import os

# Directory path
directory = '/content/CSV'

# List files in the directory
files = os.listdir(directory)
print(files)


['hiddentest_dialogue.csv', 'train.csv', 'validation.csv', 'test.csv', 'hiddentest_topic.csv']


In [9]:
from datasets import load_dataset

# Load the dataset from a CSV file
Dataset_text_summarization = load_dataset('/content/CSV')

# Inspect the first few rows of the dataset
print(Dataset_text_summarization)



Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


In [10]:
# Check available splits
print(Dataset_text_summarization)

# Calculate split lengths for the existing splits
split_lengths = [len(Dataset_text_summarization[split]) for split in Dataset_text_summarization]
print(f"Split lengths: {split_lengths}")

# Check column names in the 'train' split
print(f"Features (columns): {Dataset_text_summarization['train'].column_names}")

# Access a specific record from the 'train' split (since 'test' is not available)
print(f"\nContent of train sample 1: {Dataset_text_summarization['train'][1]['dialogue']}")

# If you had a summary column, print the corresponding summary (change column name if necessary)
print("\nSummary:")
print(Dataset_text_summarization['train'][1]["summary"])


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})
Split lengths: [12460, 500, 1500]
Features (columns): ['id', 'dialogue', 'summary', 'topic']

Content of train sample 1: #Person1#: Hello Mrs. Parker, how have you been?
#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.
#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.
#Person2#: What about Rubella and Mumps?
#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.
#Person2#: OK, great. Doctor, I t

In [11]:
def convert_example_to_features(example_batch):
    # Tokenize the input dialogue
    input_encodings = tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True,  # Truncate to 1024 tokens
        padding='max_length'  # Ensure padding to max_length
    )

    # Tokenize the target summary
    target_encodings = tokenizer(
        example_batch['summary'],
        max_length=128,
        truncation=True,  # Truncate to 128 tokens
        padding='max_length'  # Ensure padding to max_length
    )

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [12]:
# Map the function to the dataset, process in batches
dataset_text_summarization_pt = Dataset_text_summarization.map(
    convert_example_to_features,
    batched=True,  # Set batched=True to process multiple samples at once
    batch_size=16  # You can adjust the batch size as needed
)


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [13]:
dataset_text_summarization_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12460
})

In [14]:
#training
from transformers import DataCollatorForSeq2Seq
# Ensure that 'model_pegasus' is defined as your Pegasus model
seq2seq_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Pass the tokenizer
    model=model_pegasus   # Use the correct model variable (e.g., model_pegasus)
)


In [15]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-Dataset_text_summarization',  # Directory to save model and checkpoints
    num_train_epochs=1,                              # Number of training epochs
    warmup_steps=500,                                # Warmup steps for learning rate scheduler
    per_device_train_batch_size=1,                   # Batch size per device for training
    per_device_eval_batch_size=1,                    # Batch size per device for evaluation
    weight_decay=0.01,                               # Weight decay for optimizer
    logging_steps=10,                                # Log every 10 steps
    evaluation_strategy='steps',                     # Evaluate at certain steps
    eval_steps=500,                                  # Perform evaluation every 500 steps
    save_steps=500,                                  # Save checkpoint every 500 steps (adjusted from 1e6)
    gradient_accumulation_steps=16                   # Accumulate gradients over 16 steps
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model_pegasus,                               # The model you're training
    args=trainer_args,                                 # Training arguments
    tokenizer=tokenizer,                               # The tokenizer for preprocessing
    data_collator=seq2seq_data_collator,               # Data collator for batching
    train_dataset=dataset_text_summarization_pt['train'],  # Use the 'train' split for training
    eval_dataset=dataset_text_summarization_pt['validation']  # Use 'validation' split for evaluation
)


In [17]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.375000,0.339499


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=778, training_loss=3.86892330707805, metrics={'train_runtime': 8645.4445, 'train_samples_per_second': 1.441, 'train_steps_per_second': 0.09, 'total_flos': 3.5968053028061184e+16, 'train_loss': 3.86892330707805, 'epoch': 0.9990369181380417})

In [17]:
def generate_batch_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i : i + batch_size]

def calculate_metric_on_train_ds(dataset,metric,model,tokenizer,
                                 batch_size=16,device=device,
                                 column_text="article",
                                 column_summary="highlights"):
    article_batches=list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches=list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches,target_batches),total=len(article_batches)):

        input =tokenizer(article_batch, max_length=1024,truncation=True,
                         padding = 'max_length',return_tensors="pt")

        summaries= model.generate(input_ids=inputs["input_ids"].to(device),
                                  attention_mask=inputs["attention_mask"].to(device),
                                  length_penalty=0.8,num_beam=8,max_length=128)

        decoder_summaries=[tokenizer.decode(s,skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
              for s in summaries]

        decoder_summaries=[d.replace("", "") for d in decoded_summaries ]

        metric.add_batch(predictions=decode_summaries, references=target_batch)

        score=metric.compute()
        return score


In [18]:
rough_names=["rouge1","rouge2","rougel","rougeLsum"]
rouge_metric=load_metric('rouge')

In [22]:
!pip install datasets rouge_score


In [27]:
import evaluate
import torch
import pandas as pd

# Load ROUGE metric
rouge_metric = evaluate.load("rouge")

# Function to generate predictions and compute the ROUGE score
def calculate_rouge_score(dataset, model, tokenizer, batch_size, column_text, column_summary):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available, otherwise CPU
    model.to(device)  # Move the model to the correct device

    predictions, references = [], []

    # Iterate over the dataset in batches
    for i in range(0, len(dataset), batch_size):
        # Select a batch of examples using `.select()`
        batch = dataset.select(range(i, min(i + batch_size, len(dataset))))

        # Tokenize inputs and move tensors to the appropriate device
        inputs = tokenizer(batch[column_text], return_tensors='pt', truncation=True, padding=True)
        input_ids = inputs.input_ids.to(device)  # Move input tensors to the same device as the model

        # Generate predictions
        preds = model.generate(input_ids)
        decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]

        # Append predictions and references
        predictions.extend(decoded_preds)
        references.extend(batch[column_summary])

    # Compute ROUGE score
    result = rouge_metric.compute(predictions=predictions, references=references)
    return result

# Assuming you want to evaluate on a small subset of your test dataset
score = calculate_rouge_score(
    Dataset_text_summarization['test'].select(range(0, 10)),  # Use `.select()` for selecting a subset
    trainer.model,                                           # Model from your trainer
    tokenizer,                                               # Tokenizer
    batch_size=2,                                            # Batch size
    column_text='dialogue',                                  # Column with input text
    column_summary='summary'                                 # Column with target summaries
)

# Convert the ROUGE score to a DataFrame (access values directly)
rouge_dict = {rn: score[rn] for rn in score.keys()}
pd.DataFrame(rouge_dict, index=[f'pegasus'])


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.379692,0.112384,0.279676,0.278285


In [31]:
# Save the model
model_pegasus.save_pretrained("Text-Summarization")

# Save the tokenizer
tokenizer.save_pretrained("Text-Summarization")

print("Model and tokenizer saved to 'Text-Summarization'")


Model and tokenizer saved to 'Text-Summarization'


In [40]:
from transformers import AutoTokenizer

# Load the tokenizer from the specified directory
tokenizer = AutoTokenizer.from_pretrained("/content/Text-Summarization")



In [41]:
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Check the directory for model files
print(os.listdir("/content/pegasus-Dataset_text_summarization"))

# Save the model and tokenizer again if necessary
# Ensure your model has been trained or fine-tuned before running this
model_pegasus.save_pretrained("/content/pegasus-Dataset_text_summarization")
tokenizer.save_pretrained("/content/pegasus-Dataset_text_summarization")

# Load the model and tokenizer
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("/content/pegasus-Dataset_text_summarization")
tokenizer = AutoTokenizer.from_pretrained("/content/pegasus-Dataset_text_summarization")

# Load the summarization pipeline
pipe = pipeline("summarization", model=model_pegasus, tokenizer=tokenizer)

# Sample text and reference summary from the dataset
sample_text = Dataset_text_summarization["test"][0]["dialogue"]
reference = Dataset_text_summarization["test"][0]["summary"]

# Define the generation arguments
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}

# Print the dialogue (input text)
print("Dialogue:")
print(sample_text)

# Print the reference summary (actual summary)
print("\nReference Summary:")
print(reference)

# Generate a summary using the model
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])


['checkpoint-500', 'runs', 'checkpoint-778']


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Dialogue:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with th